In [3]:
import requests
import json

'''

Código que resulta em compilado_user e tweets_final, respectivamente os dados de perfil e as publicações de uma lista de usuários do Twitter.
Necessário acesso de desenvolvedor acadêmico OAuth 2.0.

'''

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer #sua chave de acesso"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

userslist = ["LulaOficial,jairbolsonaro"] #definição dos usuários a serem estudados

####### Coleta de dados dos perfis #######
##código original: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Lookup/get_users_with_bearer_token.py

user_fields = "user.fields=description,created_at,public_metrics" #definição das métricas coletadas
url_list = [] #lista vazia pra popular com as urls que vão ser consultadas

for i in userslist: #for pra criar as urls com base na lista de usuários
  usernames = "usernames="+i
  url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
  url_list.append(url)

def connect_to_endpoint1(url):
    #fazer o request com base na url e na autenticação
    response = requests.request("GET", url, auth=bearer_oauth,)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()   

lista_final = []
for i in url_list:
    #pra cada url criada com base no nome de usuário, a função de request roda e grava o resultado numa lista
    dic_parcial = json.loads((json.dumps(connect_to_endpoint1(i), indent=4, sort_keys=True)))
    lista_final.append(dic_parcial) #é criada uma lista de dicionários com a chave 'data

compilado_user = []
for i in range(len(lista_final)):
    #transformação da lista de um dicionário chave 'data' em uma lista de itens de dicionários pra cada perfil separado
    compilado_user.extend(lista_final[i]['data'])

####### Coleta das publicações #######
##código original: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Tweet-Timeline/user_tweets.py

start_time = "2022-08-16T00:00:00Z" #data inicial da coleta
campos = "created_at,author_id,lang,public_metrics,possibly_sensitive,entities,context_annotations,source,conversation_id,referenced_tweets,withheld,attachments" #métricas coletadas
metricas = {"tweet.fields": campos, "start_time":start_time, "max_results":"100"}

def connect_to_endpoint(url, params):
   #criar o request de url de ids e parametros dados, usando a autenticação
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

url_id_list = [] #lista vazia pra popular com as urls
for i in range(len(compilado_user)): #criar as urls com base na lista de usuários
  user_id = (compilado_user[i]['id'])
  url = "https://api.twitter.com/2/users/{}/tweets".format(user_id)
  url_id_list.append(url)

tweets_final = []
for i in url_id_list:
    tweets_parcial = json.loads((json.dumps(connect_to_endpoint(i,metricas), indent=4, sort_keys=True))) #lista parcial porque se bater no limite de 100 tweets precisa rodar a página seguinte
    tweets_final.append(tweets_parcial) #incluindo o resultado da coleta da id na lista final
    while (tweets_final[len(tweets_final)-1]['meta']['result_count'] == 100): #caso pegue 100 tweets, precisa verificar na página seguinte se tem mais coisa pra coletar
      metricas_complemento = {"tweet.fields": campos,"start_time":start_time, "max_results":"100", "pagination_token":tweets_final[len(tweets_final)-1]['meta']['next_token']} #usa a chave "next-token" dada pela api na coleta
      tweets_complemento =  json.loads((json.dumps(connect_to_endpoint(i,metricas_complemento), indent=4, sort_keys=True))) #coleta os complementos
      tweets_final.append(tweets_complemento) #inclui o resultado dos complementos na lista final
    else:
      print('foi') #se não tiver mais de 100, avisa só para ter controle visual do código rodando

foi
foi
